In [ ]:
from numba import jit, cuda
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# to measure exec time 
from timeit import default_timer as timer

def drawgraph(xs, ys):
    plt.plot(xs, ys, linestyle='-', linewidth = 1)
    #plt.scatter(xs, ys, 2, "#0000FF", linestyle='-')
    
# function optimized to run on gpu  
@jit(target ="cuda")
def func(xs, ts, xxs, ys, E_x, E_y, n):
    # const
    x = 0.1; y = 0.1;
    dt = 0.0001
    
    def F1(x, y):
        return x * (1 - E_x * x) - x * y

    def F2(x, y):
        return x * y * (1 - E_y * y) - y
    
    for t in range(n):
        xs[t] = x 
        #-----------------------------------------------------
        k1_x = F1(x, y)
        k1_y = F2(x, y)
        k2_x = F1(x + dt / 2 * k1_x, y + dt / 2 * k1_y)
        k2_y = F2(x + dt / 2 * k1_x, y + dt / 2 * k1_y)
        k3_x = F1(x + dt / 2 * k2_x, y + dt / 2 * k2_y)
        k3_y = F2(x + dt / 2 * k2_x, y + dt / 2 * k2_y)
        k4_x = F1(x + dt * k3_x, y + dt * k3_y)
        k4_y = F2(x + dt * k3_x, y + dt * k3_y)
        #-----------------------------------------------------
        x += dt / 6 * (k1_x + 2 * k2_x + 2 * k3_x + k4_x)
        y += dt / 6 * (k1_y + 2 * k2_y + 2 * k3_y + k4_y)
        #-----------------------------------------------------
        '''
        if x > 20 or x < -10:
            break
        if y > 20 or y < -10:
            break
        '''
        xxs[t] = x
        ys[t] = y
        ts[t] = t

if __name__=="__main__":

    n = 10000000

    xs = np.ones(n, dtype = np.float64)
    xxs = np.ones(n, dtype = np.float64)
    ys = np.ones(n, dtype = np.float64)
    ts = np.ones(n,  dtype = np.float64)
    start = timer()
    
    #for e_x in range(-10, 1, 1):
    E_x = 0.1
    E_y = -0.5
    func(xs, ts, xxs, ys, E_x, E_y, n)
    drawgraph(xs, ys)
    drawgraph(xs, xxs)
    drawgraph(ts, xs)
    drawgraph(ts, ys)
    
    print("with GPU:", timer()-start)
    plt.show()
